# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
sns.set()
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [6]:
import pandas as pd

def extract_time_series_features(data, window_size=60):
    """
    Extract rolling window features (mean, std, min, max, autocorrelation) 
    from time series physiological data.

    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data with columns like 'timestamp', 
        'heart_rate', 'eda', 'temperature'
    window_size : int
        Size of the rolling window in seconds

    Returns
    -------
    pd.DataFrame
        DataFrame with rolling window features added
    """
    # Ensure timestamp is datetime and sorted
    data = data.copy()

    # Set timestamp as index for rolling operation
    data.set_index('timestamp', inplace=True)

    # Define features to compute
    signals = ['heart_rate', 'eda', 'temperature']
    result = pd.DataFrame(index=data.index)

    for signal in signals:
        rolling = data[signal].rolling(f'{window_size}s')

        result[f'{signal}_mean'] = rolling.mean()
        result[f'{signal}_std'] = rolling.std()
        result[f'{signal}_min'] = rolling.min()
        result[f'{signal}_max'] = rolling.max()

        # autocorrelation (lag=1)
        result[f'{signal}_autocorr'] = (
            data[signal].rolling(f'{window_size}s')
            .apply(lambda x: x.autocorr(lag=1), raw=False)
        )

    # Keep the original signals for reference
    result[signals] = data[signals]

    # Reset index if needed
    result = result.reset_index()

    return result

data = pd.read_csv('/workspaces/4-it-s-about-time-RayanHSaeed/data/processed/S1_processed.csv') 
data['timestamp'] = pd.to_datetime(data['timestamp']) 
features_df = extract_time_series_features(data, window_size=60)
print(features_df.head())

/home/codespace/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3037: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/codespace/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3037: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/home/codespace/.local/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:3037: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)


            timestamp  heart_rate_mean  heart_rate_std  heart_rate_min  \
0 2018-10-13 12:56:06          102.020             NaN          102.02   
1 2018-10-13 12:56:10          102.020         0.00000          102.02   
2 2018-10-13 12:56:14          102.020         0.00000          102.02   
3 2018-10-13 12:56:16           97.515         9.01000           84.00   
4 2018-10-13 12:56:17           95.012         9.60261           84.00   

   heart_rate_max  heart_rate_autocorr  eda_mean   eda_std  eda_min   eda_max  \
0          102.02                  NaN  0.000000       NaN      0.0  0.000000   
1          102.02                  NaN  0.001282  0.001812      0.0  0.002563   
2          102.02                  NaN  0.007261  0.010436      0.0  0.019221   
3          102.02                  NaN  0.035944  0.057995      0.0  0.121993   
4          102.02             0.543954  0.053154  0.063273      0.0  0.121993   

   eda_autocorr  temperature_mean  temperature_std  temperature_min 

## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [ ]:
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

def build_arima_model(series, order=(1, 1, 1), output_dir='plots'):
    """
    Fit an ARIMA model to the time series and generate diagnostic plots.

    Parameters
    ----------
    series : pd.Series
        Time series data to model (should be indexed by timestamp)
    order : tuple
        (p, d, q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots

    Returns
    -------
    ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Drop NA values and fit model
    series = series.dropna()
    model = ARIMA(series, order=order)
    results = model.fit()

    # Model Fit vs Original
    fig1, ax1 = plt.subplots(figsize=(10, 4))
    ax1.plot(series, label='Original')
    ax1.plot(results.fittedvalues, label='Fitted', linestyle='--')
    ax1.set_title('ARIMA Model Fit')
    ax1.legend()
    fit_path = os.path.join(output_dir, 'arima_model_fit.png')
    fig1.tight_layout()
    fig1.savefig(fit_path)
    plt.close(fig1)

    # Residuals 
    residuals = results.resid
    fig2, ax2 = plt.subplots(2, 1, figsize=(10, 6))
    ax2[0].plot(residuals)
    ax2[0].set_title('Residuals')
    ax2[1].hist(residuals, bins=30)
    ax2[1].set_title('Residuals Histogram')
    resid_path = os.path.join(output_dir, 'arima_residuals.png')
    fig2.tight_layout()
    fig2.savefig(resid_path)
    plt.close(fig2)

    # Forecast 
    forecast_steps = min(50, len(series) // 2)
    forecast = results.get_forecast(steps=forecast_steps)
    pred_ci = forecast.conf_int()

    fig3, ax3 = plt.subplots(figsize=(10, 4))
    ax3.plot(series, label='Observed')
    forecast_index = range(len(series), len(series) + forecast_steps)
    ax3.plot(forecast_index, forecast.predicted_mean, label='Forecast')
    ax3.fill_between(forecast_index,
                     pred_ci.iloc[:, 0],
                     pred_ci.iloc[:, 1],
                     color='lightgrey', alpha=0.5)
    ax3.set_title(f'Forecast (next {forecast_steps} steps)')
    ax3.legend()
    forecast_path = os.path.join(output_dir, 'arima_forecast.png')
    fig3.tight_layout()
    fig3.savefig(forecast_path)
    plt.close(fig3)

    return results


series = data.set_index('timestamp')['heart_rate']
model_results = build_arima_model(series, order=(2,1,2), output_dir='plots/arima')

/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/codespace/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.